# Data Cleaning & Time Ordering

**Problem Identified:**
- Original log files may not be properly time-ordered
- This could affect incident analysis accuracy
- Need clean, chronologically sorted dataset

**Cleaning Objectives:**
1. Parse and validate all log entries
2. Sort by timestamp chronologically
3. Identify and flag any time gaps or anomalies
4. Create clean, ordered output files
5. Verify data integrity and continuity

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import os

print(" Data Cleaning Environment Ready!")
print(" Starting comprehensive data validation and ordering...")

Data Cleaning Environment Ready!
Starting comprehensive data validation and ordering...


## 1. Raw Data Inspection & Validation

In [2]:
def inspect_raw_file(filepath):
    """
    Inspect raw file for time ordering and data quality issues
    """
    print(f"\n🔍 INSPECTING: {filepath}")
    print("=" * 50)
    
    timestamps = []
    parse_errors = []
    line_count = 0
    
    # Read and extract timestamps
    with open(filepath, 'r') as file:
        for line_num, line in enumerate(file, 1):
            line_count += 1
            line = line.strip()
            
            if not line:
                continue
                
            # Extract timestamp
            pattern = r'eta_\w+\.\d+:([\d/]+) ([\d:.]+) \d+ TOOK [\d.]+s'
            match = re.match(pattern, line)
            
            if match:
                date_str, time_str = match.groups()
                try:
                    dt = datetime.strptime(f"{date_str} {time_str}", "%d/%m/%Y %H:%M:%S.%f")
                    timestamps.append({'line': line_num, 'timestamp': dt, 'raw_line': line})
                except ValueError as e:
                    parse_errors.append({'line': line_num, 'error': str(e), 'raw_line': line[:100]})
            else:
                parse_errors.append({'line': line_num, 'error': 'Pattern mismatch', 'raw_line': line[:100]})
    
    print(f" FILE STATISTICS:")
    print(f"  Total lines: {line_count:,}")
    print(f"  Valid timestamps: {len(timestamps):,}")
    print(f"  Parse errors: {len(parse_errors):,}")
    
    if timestamps:
        # Convert to DataFrame for analysis
        df_temp = pd.DataFrame(timestamps)
        
        # Check time ordering
        is_sorted = df_temp['timestamp'].is_monotonic_increasing
        
        print(f"\n TIME ORDERING:")
        print(f"  First timestamp: {df_temp['timestamp'].min()}")
        print(f"  Last timestamp: {df_temp['timestamp'].max()}")
        print(f"  Duration: {(df_temp['timestamp'].max() - df_temp['timestamp'].min()).total_seconds() / 3600:.1f} hours")
        print(f"  Chronologically sorted: {' YES' if is_sorted else ' NO'}")
        
        # Check for duplicates
        duplicates = df_temp.duplicated('timestamp').sum()
        print(f"  Duplicate timestamps: {duplicates:,}")
        
        # Check for time gaps
        df_temp = df_temp.sort_values('timestamp')
        time_diffs = df_temp['timestamp'].diff()
        large_gaps = time_diffs[time_diffs > timedelta(minutes=5)]
        
        print(f"  Large time gaps (>5min): {len(large_gaps):,}")
        
        if len(large_gaps) > 0:
            print(f"    Largest gap: {large_gaps.max()}")
        
        # Show ordering issues
        if not is_sorted:
            # Find where ordering breaks
            for i in range(1, min(len(df_temp), 1000)):
                if df_temp.iloc[i]['timestamp'] < df_temp.iloc[i-1]['timestamp']:
                    print(f"\n FIRST ORDERING ISSUE FOUND:")
                    print(f"    Line {df_temp.iloc[i-1]['line']}: {df_temp.iloc[i-1]['timestamp']}")
                    print(f"    Line {df_temp.iloc[i]['line']}: {df_temp.iloc[i]['timestamp']}")
                    break
    
    # Show parse errors (first few)
    if parse_errors:
        print(f"\n PARSE ERRORS (first 5):")
        for error in parse_errors[:5]:
            print(f"    Line {error['line']}: {error['error']}")
            print(f"      {error['raw_line']}")
    
    return timestamps, parse_errors

# Inspect both files
time6_data, time6_errors = inspect_raw_file('time6.txt')
time7_data, time7_errors = inspect_raw_file('time.txt')


INSPECTING: time6.txt
 FILE STATISTICS:
  Total lines: 141,657
  Valid timestamps: 141,657
  Parse errors: 0

⏰ TIME ORDERING:
  First timestamp: 2025-11-06 00:00:00.850000
  Last timestamp: 2025-11-06 23:59:56.860000
  Duration: 24.0 hours
  Chronologically sorted: ❌ NO
  Duplicate timestamps: 229
  Large time gaps (>5min): 0

🔍 INSPECTING: time.txt
📊 FILE STATISTICS:
  Total lines: 16,529
  Valid timestamps: 16,529
  Parse errors: 0

⏰ TIME ORDERING:
  First timestamp: 2025-11-07 00:00:00.700000
  Last timestamp: 2025-11-07 03:33:49.509000
  Duration: 3.6 hours
  Chronologically sorted: ❌ NO
  Duplicate timestamps: 21
  Large time gaps (>5min): 0


## 2. Clean Data Parsing & Validation

In [3]:
def parse_clean_logs(filepath, file_label):
    """
    Parse logs with comprehensive validation and cleaning
    """
    print(f"\n CLEANING: {filepath} ({file_label})")
    
    clean_records = []
    validation_issues = []
    
    with open(filepath, 'r') as file:
        for line_num, line in enumerate(file, 1):
            line = line.strip()
            if not line:
                continue
                
            # Full pattern matching
            pattern = r'(eta_\w+)\.(\d+):([\d/]+) ([\d:.]+) (\d+) TOOK ([\d.]+)s'
            match = re.match(pattern, line)
            
            if match:
                agent_type, pid, date_str, time_str, transaction_id, execution_time = match.groups()
                
                try:
                    # Parse and validate all components
                    dt = datetime.strptime(f"{date_str} {time_str}", "%d/%m/%Y %H:%M:%S.%f")
                    pid_int = int(pid)
                    txn_id = int(transaction_id)
                    exec_time = float(execution_time)
                    
                    # Validation checks
                    issues = []
                    
                    # Check for reasonable values
                    if exec_time < 0:
                        issues.append("Negative execution time")
                    elif exec_time > 300:  # >5 minutes seems suspicious
                        issues.append(f"Extremely long execution time: {exec_time}s")
                    
                    if pid_int <= 0:
                        issues.append("Invalid PID")
                    
                    if txn_id <= 0:
                        issues.append("Invalid transaction ID")
                    
                    # Check date reasonableness (should be Nov 6-7, 2025)
                    if not (datetime(2025, 11, 6) <= dt <= datetime(2025, 11, 8)):
                        issues.append(f"Date outside expected range: {dt.date()}")
                    
                    # Store record with metadata
                    record = {
                        'source_file': file_label,
                        'line_number': line_num,
                        'agent_type': agent_type,
                        'pid': pid_int,
                        'datetime': dt,
                        'transaction_id': txn_id,
                        'execution_time': exec_time,
                        'raw_line': line,
                        'validation_issues': issues,
                        'is_valid': len(issues) == 0
                    }
                    
                    clean_records.append(record)
                    
                    if issues:
                        validation_issues.extend([{
                            'line': line_num, 
                            'issue': issue, 
                            'record': record
                        } for issue in issues])
                        
                except Exception as e:
                    validation_issues.append({
                        'line': line_num,
                        'issue': f"Parse error: {str(e)}",
                        'raw_line': line
                    })
    
    # Convert to DataFrame
    df = pd.DataFrame(clean_records)
    
    print(f"  Parsed records: {len(clean_records):,}")
    print(f"  Valid records: {df['is_valid'].sum():,}")
    print(f"  Records with issues: {(~df['is_valid']).sum():,}")
    print(f"  Total validation issues: {len(validation_issues):,}")
    
    return df, validation_issues

# Parse both files with cleaning
df_time6_clean, time6_issues = parse_clean_logs('time6.txt', 'time6.txt')
df_time7_clean, time7_issues = parse_clean_logs('time.txt', 'time.txt')


 CLEANING: time6.txt (time6.txt)
  Parsed records: 141,657
  Valid records: 141,657
  Records with issues: 0
  Total validation issues: 0

 CLEANING: time.txt (time.txt)
  Parsed records: 16,529
  Valid records: 16,529
  Records with issues: 0
  Total validation issues: 0


## 3. Combine and Sort Chronologically

In [4]:
# Combine datasets
print("\n COMBINING DATASETS")
print("=" * 30)

# Combine all records
df_combined = pd.concat([df_time6_clean, df_time7_clean], ignore_index=True)

print(f"Combined dataset size: {len(df_combined):,} records")
print(f"Time6.txt: {len(df_time6_clean):,} records")
print(f"Time.txt: {len(df_time7_clean):,} records")

# Sort chronologically
print(f"\n SORTING CHRONOLOGICALLY...")
df_sorted = df_combined.sort_values('datetime').reset_index(drop=True)

# Verify sorting
print(f" Chronological sorting completed")
print(f" First record: {df_sorted['datetime'].iloc[0]}")
print(f" Last record: {df_sorted['datetime'].iloc[-1]}")
print(f" Total duration: {(df_sorted['datetime'].iloc[-1] - df_sorted['datetime'].iloc[0]).total_seconds() / 3600:.1f} hours")

# Check for overlaps between files
if len(df_time6_clean) > 0 and len(df_time7_clean) > 0:
    time6_max = df_time6_clean['datetime'].max()
    time7_min = df_time7_clean['datetime'].min()
    
    print(f"\n FILE BOUNDARY ANALYSIS:")
    print(f"  Time6.txt last record: {time6_max}")
    print(f"  Time.txt first record: {time7_min}")
    
    if time6_max > time7_min:
        overlap_duration = (time6_max - time7_min).total_seconds() / 60
        print(f" Time overlap detected: {overlap_duration:.1f} minutes")
        
        # Check for duplicate transactions in overlap
        overlap_records = df_sorted[
            (df_sorted['datetime'] >= time7_min) &
            (df_sorted['datetime'] <= time6_max)
        ]
        overlap_from_time6 = overlap_records[overlap_records['source_file'] == 'time6.txt']
        overlap_from_time7 = overlap_records[overlap_records['source_file'] == 'time.txt']
        
        print(f"    Records from time6.txt in overlap: {len(overlap_from_time6)}")
        print(f"    Records from time.txt in overlap: {len(overlap_from_time7)}")
        
        # Check for exact duplicates
        duplicates = df_sorted.duplicated(['datetime', 'pid', 'transaction_id'], keep=False)
        if duplicates.sum() > 0:
            print(f" Duplicate transactions found: {duplicates.sum()}")
        else:
            print(f" No duplicate transactions found")
    else:
        gap_duration = (time7_min - time6_max).total_seconds() / 60
        print(f" Time gap between files: {gap_duration:.1f} minutes")


 COMBINING DATASETS
Combined dataset size: 158,186 records
Time6.txt: 141,657 records
Time.txt: 16,529 records

 SORTING CHRONOLOGICALLY...
 Chronological sorting completed
 First record: 2025-11-06 00:00:00.850000
 Last record: 2025-11-07 03:33:49.509000
 Total duration: 27.6 hours

 FILE BOUNDARY ANALYSIS:
 Time6.txt last record: 2025-11-06 23:59:56.860000
 Time.txt first record: 2025-11-07 00:00:00.700000
 Time gap between files: 0.1 minutes


## 4. Data Quality Assessment

In [5]:
print("\n DATA QUALITY ASSESSMENT")
print("=" * 40)

# Overall statistics
valid_records = df_sorted[df_sorted['is_valid']]
invalid_records = df_sorted[~df_sorted['is_valid']]

print(f" DATASET OVERVIEW:")
print(f"  Total records: {len(df_sorted):,}")
print(f"  Valid records: {len(valid_records):,} ({len(valid_records)/len(df_sorted)*100:.1f}%)")
print(f"  Invalid records: {len(invalid_records):,} ({len(invalid_records)/len(df_sorted)*100:.1f}%)")

# Time continuity analysis
print(f"\n TIME CONTINUITY:")
if len(valid_records) > 1:
    time_diffs = valid_records['datetime'].diff().dropna()
    
    print(f"  Average time between records: {time_diffs.mean().total_seconds():.1f} seconds")
    print(f"  Median time between records: {time_diffs.median().total_seconds():.1f} seconds")
    print(f"  Maximum gap: {time_diffs.max().total_seconds() / 60:.1f} minutes")
    
    # Identify significant gaps
    large_gaps = time_diffs[time_diffs > timedelta(minutes=5)]
    if len(large_gaps) > 0:
        print(f"   Large gaps (>5min): {len(large_gaps)}")
        print(f"   Largest gap: {large_gaps.max().total_seconds() / 60:.1f} minutes")
    else:
        print(f"   No significant gaps detected")

# Performance data quality
if len(valid_records) > 0:
    exec_times = valid_records['execution_time']
    
    print(f"\n PERFORMANCE DATA QUALITY:")
    print(f"  Min execution time: {exec_times.min():.3f}s")
    print(f"  Max execution time: {exec_times.max():.1f}s")
    print(f"  Mean execution time: {exec_times.mean():.3f}s")
    print(f"  Median execution time: {exec_times.median():.3f}s")
    
    # Check for anomalies
    extremely_fast = (exec_times < 0.01).sum()
    extremely_slow = (exec_times > 120).sum()
    
    print(f"  Extremely fast (<0.01s): {extremely_fast:,}")
    print(f"  Extremely slow (>2min): {extremely_slow:,}")

# Agent and PID analysis
if len(valid_records) > 0:
    print(f"\n SYSTEM COMPONENTS:")
    print(f"  Unique agent types: {valid_records['agent_type'].nunique()}")
    print(f"  Agent type distribution: {valid_records['agent_type'].value_counts().to_dict()}")
    print(f"  Unique PIDs: {valid_records['pid'].nunique()}")
    print(f"  PID range: {valid_records['pid'].min()} - {valid_records['pid'].max()}")

# Show validation issues summary
if time6_issues or time7_issues:
    all_issues = time6_issues + time7_issues
    print(f"\n VALIDATION ISSUES SUMMARY:")
    
    # Group issues by type
    issue_types = {}
    for issue in all_issues:
        issue_type = issue['issue']
        if issue_type not in issue_types:
            issue_types[issue_type] = 0
        issue_types[issue_type] += 1
    
    for issue_type, count in sorted(issue_types.items()):
        print(f"    {issue_type}: {count:,} occurrences")


🔍 DATA QUALITY ASSESSMENT
 DATASET OVERVIEW:
  Total records: 158,186
  Valid records: 158,186 (100.0%)
  Invalid records: 0 (0.0%)

 TIME CONTINUITY:
 Average time between records: 0.6 seconds
 Median time between records: 0.4 seconds
 Maximum gap: 0.4 minutes
 No significant gaps detected

 PERFORMANCE DATA QUALITY:
  Min execution time: 0.019s
  Max execution time: 119.2s
  Mean execution time: 3.692s
  Median execution time: 1.648s
  Extremely fast (<0.01s): 0
  Extremely slow (>2min): 0

 SYSTEM COMPONENTS:
  Unique agent types: 2
  Agent type distribution: {'eta_agent': 148122, 'eta_iagent': 10064}
  Unique PIDs: 70
  PID range: 7690 - 8216


## 5. Export Clean Dataset

In [6]:
print("\n EXPORTING CLEAN DATASET")
print("=" * 35)

# Create clean subset (only valid records)
df_clean_final = valid_records[[
    'datetime', 'agent_type', 'pid', 'transaction_id', 'execution_time', 
    'source_file', 'line_number'
]].copy()

# Add derived fields for analysis
df_clean_final['date'] = df_clean_final['datetime'].dt.date
df_clean_final['hour'] = df_clean_final['datetime'].dt.hour
df_clean_final['minute'] = df_clean_final['datetime'].dt.minute
df_clean_final['is_slow'] = df_clean_final['execution_time'] > 20
df_clean_final['is_very_slow'] = df_clean_final['execution_time'] > 30
df_clean_final['is_critical'] = df_clean_final['execution_time'] > 60

# Export to CSV for further analysis
clean_filename = 'cleaned_eta_logs.csv'
df_clean_final.to_csv(clean_filename, index=False)

print(f" Clean dataset exported: {clean_filename}")
print(f"  Records: {len(df_clean_final):,}")
print(f"  Columns: {len(df_clean_final.columns)}")
print(f"  File size: {os.path.getsize(clean_filename) / 1024 / 1024:.1f} MB")

# Export validation report
validation_report = {
    'total_raw_records': len(df_combined),
    'valid_records': len(valid_records),
    'invalid_records': len(invalid_records),
    'time_range': {
        'start': str(df_clean_final['datetime'].min()),
        'end': str(df_clean_final['datetime'].max()),
        'duration_hours': (df_clean_final['datetime'].max() - df_clean_final['datetime'].min()).total_seconds() / 3600
    },
    'data_sources': {
        'time6_records': len(df_time6_clean),
        'time7_records': len(df_time7_clean)
    }
}

print(f"\n FINAL DATASET SUMMARY:")
for key, value in validation_report.items():
    if isinstance(value, dict):
        print(f"  {key}:")
        for subkey, subvalue in value.items():
            print(f"    {subkey}: {subvalue}")
    else:
        print(f"  {key}: {value:,}" if isinstance(value, int) else f"  {key}: {value}")

print(f"\n DATA CLEANING COMPLETE")
print(f"   Clean dataset ready for incident analysis")
print(f"   Use '{clean_filename}' for all subsequent analysis")
print(f"   Data quality: {len(valid_records)/len(df_combined)*100:.1f}% valid records")


 EXPORTING CLEAN DATASET
 Clean dataset exported: cleaned_eta_logs.csv
  Records: 158,186
  Columns: 13
  File size: 16.7 MB

 FINAL DATASET SUMMARY:
  total_raw_records: 158,186
  valid_records: 158,186
  invalid_records: 0
  time_range:
    start: 2025-11-06 00:00:00.850000
    end: 2025-11-07 03:33:49.509000
    duration_hours: 27.56351638888889
  data_sources:
    time6_records: 141657
    time7_records: 16529

 DATA CLEANING COMPLETE!
   Clean dataset ready for incident analysis
   Use 'cleaned_eta_logs.csv' for all subsequent analysis
   Data quality: 100.0% valid records


## 6. Quick Incident Timeline Verification

In [7]:
# Quick verification of incident timeframe with clean data
print("\n INCIDENT TIMELINE VERIFICATION WITH CLEAN DATA")
print("=" * 55)

incident_time = datetime(2025, 11, 6, 22, 10)
analysis_start = datetime(2025, 11, 6, 21, 30)
analysis_end = datetime(2025, 11, 7, 3, 30)

# Filter to analysis window
analysis_data = df_clean_final[
    (df_clean_final['datetime'] >= analysis_start) &
    (df_clean_final['datetime'] <= analysis_end)
].copy()

print(f" ANALYSIS WINDOW DATA:")
print(f"  Records in analysis window: {len(analysis_data):,}")
print(f"  Time range: {analysis_data['datetime'].min()} → {analysis_data['datetime'].max()}")

if len(analysis_data) > 0:
    # Hour-by-hour breakdown
    hourly = analysis_data.groupby(analysis_data['datetime'].dt.floor('H')).agg({
        'execution_time': ['count', 'mean'],
        'is_slow': 'sum',
        'is_critical': 'sum'
    }).round(2)
    
    hourly.columns = ['count', 'avg_time', 'slow_count', 'critical_count']
    hourly['slow_pct'] = (hourly['slow_count'] / hourly['count'] * 100).round(1)
    
    print(f"\n HOURLY BREAKDOWN (clean data):")
    for hour, row in hourly.iterrows():
        print(f"  {hour.strftime('%H:00')}: {row['count']:>4} txns, {row['avg_time']:>5.1f}s avg, {row['slow_pct']:>4.1f}% slow")
    
    # Identify peak issues
    worst_hour = hourly['avg_time'].idxmax()
    highest_slow_rate = hourly['slow_pct'].idxmax()
    
    print(f"\n CLEAN DATA INSIGHTS:")
    print(f"  Worst performance: {worst_hour.strftime('%H:00')} ({hourly.loc[worst_hour, 'avg_time']:.1f}s avg)")
    print(f"  Highest slow rate: {highest_slow_rate.strftime('%H:00')} ({hourly.loc[highest_slow_rate, 'slow_pct']:.1f}%)")
    
    # Critical transactions
    critical_txns = analysis_data[analysis_data['is_critical']]
    if len(critical_txns) > 0:
        print(f"  Critical transactions (>60s): {len(critical_txns):,}")
        worst_txn = critical_txns.loc[critical_txns['execution_time'].idxmax()]
        print(f"  Worst transaction: {worst_txn['execution_time']:.1f}s at {worst_txn['datetime'].strftime('%H:%M:%S')}")

print(f"\n Clean data ready for accurate incident analysis!")


 INCIDENT TIMELINE VERIFICATION WITH CLEAN DATA
 ANALYSIS WINDOW DATA:
  Records in analysis window: 30,818
  Time range: 2025-11-06 21:30:04.074000 → 2025-11-07 03:29:59.972000

 HOURLY BREAKDOWN (clean data):
  21:00: 2756.0 txns,   2.0s avg,  0.4% slow
  22:00: 5896.0 txns,  12.1s avg, 15.1% slow
  23:00: 5887.0 txns,  10.8s avg, 10.8% slow
  00:00: 5777.0 txns,  10.5s avg,  9.9% slow
  01:00: 4644.0 txns,   5.0s avg,  3.4% slow
  02:00: 3981.0 txns,   1.8s avg,  0.2% slow
  03:00: 1877.0 txns,   1.1s avg,  0.0% slow

 CLEAN DATA INSIGHTS:
  Worst performance: 22:00 (12.1s avg)
  Highest slow rate: 22:00 (15.1%)
  Critical transactions (>60s): 179
  Worst transaction: 115.9s at 22:25:27

 Clean data ready for accurate incident analysis!


/tmp/nix-shell.aQbSAD/ipykernel_63401/1455007149.py:21: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly = analysis_data.groupby(analysis_data['datetime'].dt.floor('H')).agg({
